In [1]:
!pip install jax[tpu] -f https://storage.googleapis.com/jax-releases/libtpu_releases.html

Looking in links: https://storage.googleapis.com/jax-releases/libtpu_releases.html
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
     |████████████████████████████████| 1.3 MB 4.9 MB/s 
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - \ | done
    Preparing wheel metadata ... - \ | / done
     |████████████████████████████████| 1.0 MB 85.4 MB/s 
     |████████████████████████████████| 17.3 MB 84.7 MB/s 
     |████████████████████████████████| 65 kB 5.0 MB/s 
     |████████████████████████████████| 34.5 MB 92.6 MB/s 
     |████████████████████████████████| 71.6 MB 51 kB/

In [2]:
import jax
jax.__version__

'0.4.13'

In [48]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

tpu


## DeviceArray properties

In [3]:
import numpy as np
import jax.numpy as jnp

In [4]:
np.array([1, 42, 31337])

array([    1,    42, 31337])

In [5]:
jnp.array([1, 42, 31337])

Array([    1,    42, 31337], dtype=int32)

In [6]:
np.sum([1, 42, 31337])

31380

In [7]:
try:
  jnp.sum([1, 42, 31337])
except TypeError as e:
  print(e)

sum requires ndarray or scalar arguments, got <class 'list'> at position 0.


In [8]:
jnp.sum(jnp.array([1, 42, 31337]))

Array(31380, dtype=int32)

In [9]:
arr = jnp.array([1, 42, 31337])

In [10]:
arr.ndim

1

In [11]:
arr.shape

(3,)

In [12]:
arr.dtype

dtype('int32')

In [13]:
arr.size

3

In [14]:
arr.nbytes

12

## Devices

In [15]:
import jax

In [16]:
jax.devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

In [17]:
jax.devices('cpu')

[CpuDevice(id=0)]

In [19]:
jax.device_count('tpu')

8

In [20]:
jax.local_devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

In [21]:
arr = jnp.array([1, 42, 31337])

In [22]:
arr.device()

TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)

In [23]:
arr_cpu = jax.device_put(arr, jax.devices('cpu')[0])

In [24]:
arr_cpu.device()

CpuDevice(id=0)

In [25]:
arr.device()

TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)

In [26]:
arr_host = jax.device_get(arr)

In [27]:
type(arr_host)

numpy.ndarray

In [28]:
arr_host

array([    1,    42, 31337], dtype=int32)

In [29]:
arr + arr_cpu

Array([    2,    84, 62674], dtype=int32)

In [30]:
arr_tpu = jax.device_put(arr, jax.devices('tpu')[0])

In [31]:
arr_tpu.device()

TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)

In [32]:
try:
  arr_tpu + arr_cpu
except ValueError as e:
  print(e)

Received incompatible devices for jitted computation. Got argument x1 of jax.numpy.add with shape int32[3] and device ids [0] on platform TPU and argument x2 of jax.numpy.add with shape int32[3] and device ids [0] on platform CPU


## Asyncronous dispatch

In [33]:
import jax

In [34]:
a = jnp.array(range(1000000)).reshape((1000,1000))

In [35]:
a.shape

(1000, 1000)

In [36]:
a.device()

TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)

In [55]:
%time x = jnp.dot(a,a)

CPU times: user 1.07 ms, sys: 674 µs, total: 1.74 ms
Wall time: 953 µs


In [56]:
%time x = jnp.dot(a,a).block_until_ready()

CPU times: user 1.85 ms, sys: 1.17 ms, total: 3.02 ms
Wall time: 2.07 ms


In [57]:
%time x = np.asarray(jnp.dot(a,a))

CPU times: user 5.43 ms, sys: 638 µs, total: 6.07 ms
Wall time: 3.77 ms


In [58]:
a_cpu = jax.device_put(a, jax.devices('cpu')[0])

In [59]:
a_cpu.device()

CpuDevice(id=0)

In [61]:
%time x = jnp.dot(a_cpu,a_cpu).block_until_ready()

CPU times: user 341 ms, sys: 29.6 ms, total: 371 ms
Wall time: 5.19 ms


## Immutability

In [65]:
import numpy as np
import jax.numpy as jnp

In [66]:
a_jnp = jnp.array(range(10))
a_np  = np.array(range(10))

In [67]:
a_jnp

Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)

In [68]:
a_np[5], a_jnp[5]

(5, Array(5, dtype=int32))

In [69]:
a_np[5] = 100

In [70]:
a_np[5]

100

In [71]:
try:
  a_jnp[5] = 100
except TypeError as e:
  print(e)

'<class 'jaxlib.xla_extension.ArrayImpl'>' object does not support item assignment. JAX arrays are immutable. Instead of ``x[idx] = y``, use ``x = x.at[idx].set(y)`` or another .at[] method: https://jax.readthedocs.io/en/latest/_autosummary/jax.numpy.ndarray.at.html


In [72]:
a_jnp = a_jnp.at[5].set(100)

In [73]:
a_jnp[5]

Array(100, dtype=int32)

In [74]:
a_jnp[42]

Array(9, dtype=int32)

In [75]:
a_jnp.at[42].get()

Array(9, dtype=int32)

In [76]:
a_jnp.at[42].get(mode='clip')

Array(9, dtype=int32)

In [77]:
a_jnp.at[42].get(mode='drop')

Array(-2147483648, dtype=int32)

In [78]:
a_jnp.at[42].get(mode='fill', fill_value=-1)

Array(-1, dtype=int32)

In [79]:
a_jnp = a_jnp.at[42].set(100)
a_jnp

Array([  0,   1,   2,   3,   4, 100,   6,   7,   8,   9], dtype=int32)

In [80]:
a_jnp = a_jnp.at[42].set(100, mode='clip')
a_jnp

Array([  0,   1,   2,   3,   4, 100,   6,   7,   8, 100], dtype=int32)

## Working with float64

In [81]:
# this only works on startup!
from jax.config import config
config.update("jax_enable_x64", True)

In [82]:
import jax
import jax.numpy as jnp

In [83]:
# this will not work on TPU backend. Try using CPU or GPU.
x = jnp.array(range(10), dtype=jnp.float64)
x.dtype

dtype('float64')

In [84]:
x.device()

TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)

In [85]:
xc = jax.device_put(x, jax.devices('cpu')[0])

In [86]:
xc.device()

CpuDevice(id=0)

In [87]:
xc.dtype

dtype('float64')

In [88]:
xb16 = jnp.array(range(10), dtype=jnp.bfloat16)
xb16.dtype

dtype(bfloat16)

In [89]:
xb16.nbytes

20

In [90]:
x16 = jnp.array(range(10), dtype=jnp.float16)
x16.dtype

dtype('float16')

In [91]:
x16.nbytes

20

In [92]:
xb16+x16

Array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18.], dtype=float32)

In [93]:
xb16+xb16

Array([0, 2, 4, 6, 8, 10, 12, 14, 16, 18], dtype=bfloat16)

## jax.numpy & jax.lax

In [94]:
config.update("jax_enable_x64", False)

In [95]:
import jax.numpy as jnp
from jax import lax
from jax import random

In [96]:
jnp.add(42, 42.0)

Array(84., dtype=float32, weak_type=True)

In [97]:
jnp.add(42.0, 42.0)

Array(84., dtype=float32, weak_type=True)

In [98]:
try:
   lax.add(42, 42.0)
except TypeError as e:
  print(e)

lax.add requires arguments to have the same dtypes, got int32, float32. (Tip: jnp.add is a similar function that does automatic type promotion on inputs).


In [99]:
lax.add(jnp.float32(42), 42.0)

Array(84., dtype=float32)

In [100]:
def random_augmentation(image, augmentations, rng_key):
   '''A function that applies a random transformation to an image'''
   augmentation_index = random.randint(key=rng_key, minval=0, maxval=len(augmentations), shape=())
   augmented_image = lax.switch(augmentation_index, augmentations, image)
   return augmented_image

In [101]:
add_noise_func = lambda x: x+10
horizontal_flip_func = lambda x: x+1
rotate_func = lambda x: x+2
adjust_colors_func = lambda x: x+3

augmentations = [
   add_noise_func,
   horizontal_flip_func,
   rotate_func,
   adjust_colors_func
]


In [102]:
image = jnp.array(range(100))

In [103]:
random_augmentation(image, augmentations, random.PRNGKey(211))

Array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100], dtype=int32)